First, I open and explore the datasets to try to resolve any inconsistencies before beginning to analyze the data.

In [4]:
import pandas as pd

In [6]:
politicians_raw = pd.read_csv("../raw_data/politicians_by_country_AUG.2024.csv")
population_raw = pd.read_csv("../raw_data/population_by_country_AUG.2024.csv")

Often, when working with geographic data, it's challenging to perform joins because regions have different names in different datasets. Let's check if that's the case here.

In [7]:
print(set(politicians_raw["country"]) - set(population_raw["Geography"]))
print(set(population_raw["Geography"]) - set(politicians_raw["country"]))

{'Korea, South', 'Korean', 'Guinea-Bissau'}
{'Sao Tome and Principe', 'Kiribati', 'eSwatini', 'Puerto Rico', 'NORTHERN AMERICA', 'SOUTHERN AFRICA', 'Mexico', 'SOUTHEAST ASIA', 'SOUTH AMERICA', 'Philippines', 'LATIN AMERICA AND THE CARIBBEAN', 'Nauru', 'NORTHERN AFRICA', 'Reunion', 'CENTRAL ASIA', 'WESTERN EUROPE', 'Fiji', 'Canada', 'Georgia', 'Korea (North)', 'Australia', 'Mauritius', 'Guam', 'United States', 'United Kingdom', 'San Marino', 'Iceland', 'EAST ASIA', 'Ireland', 'WORLD', 'NORTHERN EUROPE', 'French Guiana', 'Suriname', 'WESTERN ASIA', 'French Polynesia', 'Curacao', 'AFRICA', 'Andorra', 'EASTERN AFRICA', 'Guadeloupe', 'China (Hong Kong SAR)', 'EUROPE', 'Brunei', 'China (Macao SAR)', 'Martinique', 'GuineaBissau', 'New Caledonia', 'CARIBBEAN', 'Denmark', 'Jamaica', 'ASIA', 'CENTRAL AMERICA', 'MIDDLE AFRICA', 'Korea (South)', 'Liechtenstein', 'Palau', 'Western Sahara', 'Mayotte', 'Netherlands', 'SOUTHERN EUROPE', 'EASTERN EUROPE', 'New Zealand', 'WESTERN AFRICA', 'Dominica', 'O

In [12]:
politicians_cleaned = politicians_raw.copy()
politicians_cleaned["country"] = politicians_cleaned["country"].replace(
    {
        "Korea, South": "Korea (South)",
        "Korean": "Korea (South)"
    }
)

population_cleaned = population_raw.copy()
population_cleaned["Geography"] = population_cleaned["Geography"].replace(
    {
        "GuineaBissau": "Guinea-Bissau"
    }
)

print(set(politicians_cleaned["country"]) - set(population_cleaned["Geography"]))


set()
